Learnings 
- put the notebook in the same directory as the keras model 
- make sure the computer is in the same directory 'cd coverted_keras'

Progress 
when runs camera opens 
compares the video to the model supplied
4 probabilities in a nparray 
probabilities per? time interval 

Start
input from camera - not taking in/storing
key input to start the game - text onto screen  
countdown to game - text onto the screen
takes in several images from the camera ?time period (4 probabilities)
?takes and average of all of the probabilities to determine what my answer was
computer randomly chooses an answer 
compares the answer to mine to see who has won 
display who has won 

Qs
how will computer know which one you have chosen
how to get text onto screen
how to create countdown timer 
why imshow at the end 

In [2]:
import cv2
from keras.models import load_model
import numpy as np
import random
import time
model = load_model('upstairs_keras_model.h5')
cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32) #1 image per iteration, 224 pixels x 224, RGB channels
started = False
play = False
capture = False

def compare_predictions(user_prediction, computer_prediction):
    if computer_prediction == 'ROCK':
        if user_prediction == 'ROCK':               
            return "it's a draw"
        elif user_prediction == 'PAPER':
            return "well done.. you win"
        elif user_prediction == 'SCISSORS':
            return "tough luck you lose"
        else:
            return "you didn't make a guess"
    elif computer_prediction == 'PAPER':
        if user_prediction == 'ROCK':
            return "tough luck you lose"
        elif user_prediction == 'PAPER':
            return "it's a draw"
        elif user_prediction == 'SCISSORS':
            return "well done.. you win"
        else:
            return "you didn't make a guess"
    elif computer_prediction == 'SCISSORS':
        if user_prediction == 'ROCK':
            return "well done.. you win"
        elif user_prediction == 'PAPER':
            return "tough luck you lose"
        elif user_prediction == 'SCISSORS':
            return "it's a draw"
        else:
            return "you didn't make a guess"
   
def predict_cam(prediction):
    if prediction[0][0] > 0.5:
        user_prediction = 'ROCK'
    elif prediction[0][1] > 0.5:
        user_prediction = 'PAPER'
    elif prediction[0][2] > 0.5:
        user_prediction = 'SCISSORS'
    else:
        user_prediction = 'NONE'
    return user_prediction

while True: 
    ret, frame = cap.read() #capturing from the camera 
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame) #creating an array of the image 
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image ?why /127 
    data[0] = normalized_image
    txt = " "
    prediction = model.predict(data)  #keras - uses model to predict from input- 4 probablities of variables 
    computer = random.choice(['ROCK','PAPER', 'SCISSORS'])
    user_prediction = predict_cam(prediction)
    computer_score = 0
    user_score = 0
    draw = 0
    not_working = 0

    if not started: 
        txt = "press a to start"
        
    if cv2.waitKey(1) & 0xFF == ord('a'): 
        if not started:
            started = True
            t0=time.time()
            
    if started:
        time_elapsed = time.time() - t0 
        count_down = 5 - time_elapsed

        if play:
            if capture == False:
                result = compare_predictions(user_prediction, computer)
                txt = f"result: {result}"
                if result == "well done.. you win":
                    user_score += 1
                elif result == "tough luck you lose":
                    computer_score += 1
                elif result == "it's a draw":
                    draw += 1
                else:
                    not_working += 1
                print("user score: ", user_score)
                print("computer score: ", computer_score)
                print("draw: ", draw)
                print("Not working: ", not_working)
                capture = True
            else:
               txt = f"result: {result}"
        else:
            txt = ''

        if count_down < -4:
            play = False
            txt = 'press s to reset'
            #txt = f'Score you: {user_score} computer:{computer_score} to you; press x to play again'
            if cv2.waitKey(1) & 0xFF == ord('s'):
                started = False

        if count_down >= 0:
            txt = f'Make your move in {int(count_down)} seconds'

        if count_down < 0:
            play = True
    
    if user_score == 3:
        break

    if computer_score == 3:
        break

    cv2.putText(frame, txt, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_4)
    cv2.imshow('frame', frame) 

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()

user score:  1
computer score:  0
draw:  0
Not working:  0
